# Mock CRISI BCA – Reviewer Analysis

This notebook reviews the applicant-submitted BCA workbook and evaluates methodological consistency with OMB Circular A-4 and A-94 guidance.

All data are synthetic and used solely for demonstration.

In [2]:
import pandas as pd
import numpy as np

applicant_file = "applicant_submission.xlsx"

inputs = pd.read_excel(applicant_file, sheet_name="Inputs")
summary = pd.read_excel(applicant_file, sheet_name="Summary")
cashflows = pd.read_excel(applicant_file, sheet_name="CashFlows")

In [3]:
pv_benefit_app = summary.loc[summary["Metric"] == "PV Benefits", "Value"].values[0]
pv_cost_app = summary.loc[summary["Metric"] == "PV Costs", "Value"].values[0]
bcr_app = summary.loc[summary["Metric"] == "BCR (7% real)", "Value"].values[0]

pv_benefit_app, pv_cost_app, bcr_app

(np.float64(210465695.9417896),
 np.float64(97891021.36827423),
 np.float64(2.15))

## Identified Methodological Issues

The following issues were identified during review:

1. Aggressive traffic growth assumption (4% annually).
2. Use of outdated Value of Statistical Life (VSL).
3. Uniform safety benefit stream not sensitivity-tested.
4. Lack of alternative discount rate (3%) analysis.

In [4]:
# Reviewer adjustment assumptions
revised_growth_rate = 0.015
discount_rate = 0.07  # confirm 7% real

# Rebuild benefit streams using revised growth
cashflows_rev = cashflows.copy()

for col, base_value in [
    ("travel_time_benefit", cashflows.loc[1, "travel_time_benefit"]),
    ("voc_benefit", cashflows.loc[1, "voc_benefit"]),
    ("emissions_benefit", cashflows.loc[1, "emissions_benefit"]),
]:
    for t in cashflows_rev["year"]:
        if t >= 1:
            cashflows_rev.loc[cashflows_rev["year"] == t, col] = \
                base_value * ((1 + revised_growth_rate) ** (t - 1))

In [5]:
cashflows_rev["safety_benefit"] = cashflows_rev["safety_benefit"] * 1.10

In [6]:
cashflows_rev["total_benefit"] = (
    cashflows_rev["safety_benefit"]
    + cashflows_rev["travel_time_benefit"]
    + cashflows_rev["voc_benefit"]
    + cashflows_rev["emissions_benefit"]
)

cashflows_rev["pv_benefit"] = cashflows_rev["total_benefit"] * cashflows_rev["discount_factor"]

pv_benefit_rev = cashflows_rev["pv_benefit"].sum()
pv_cost_rev = cashflows_rev["pv_cost"].sum()

bcr_rev = pv_benefit_rev / pv_cost_rev

pv_benefit_rev, pv_cost_rev, bcr_rev

(np.float64(197867021.84261814),
 np.float64(97891021.36827423),
 np.float64(2.0212989820407103))

In [7]:
print("Applicant BCR (7%): {:.2f}".format(bcr_app))
print("Revised BCR (7%):   {:.2f}".format(bcr_rev))

Applicant BCR (7%): 2.15
Revised BCR (7%):   2.02


In [8]:
cashflows_rev["discount_factor_3"] = 1 / ((1 + 0.03) ** cashflows_rev["year"])
cashflows_rev["pv_benefit_3"] = cashflows_rev["total_benefit"] * cashflows_rev["discount_factor_3"]
cashflows_rev["pv_cost_3"] = cashflows_rev["total_cost"] * cashflows_rev["discount_factor_3"]

bcr_3 = cashflows_rev["pv_benefit_3"].sum() / cashflows_rev["pv_cost_3"].sum()
bcr_3

np.float64(2.6548206767854556)

In [9]:
print("Revised BCR (3%):   {:.2f}".format(bcr_3))

Revised BCR (3%):   2.65


## Summary of Reviewer Adjustments

| Category | Applicant Assumption | Reviewer Adjustment | Rationale |
|-----------|---------------------|--------------------|------------|
| Traffic Growth | 4% annually | 1.5% annually | Growth not supported by corridor documentation |
| Value of Statistical Life | $9.6M | Updated per current USDOT guidance | Alignment with current federal valuation |
| Discount Rate Sensitivity | 7% only | Added 3% sensitivity | A-94 recommended sensitivity analysis |

In [10]:
print("---- Reviewer Summary ----")
print("Applicant BCR (7%): {:.2f}".format(bcr_app))
print("Revised BCR (7%):   {:.2f}".format(bcr_rev))
print("Revised BCR (3%):   {:.2f}".format(bcr_3))

---- Reviewer Summary ----
Applicant BCR (7%): 2.15
Revised BCR (7%):   2.02
Revised BCR (3%):   2.65


## Reviewer Conclusion

After adjusting unsupported growth assumptions and aligning valuation inputs with current federal guidance, the project remains economically justified under both 7% and 3% real discount rates. However, the magnitude of benefits is sensitive to traffic growth projections.

Further documentation of corridor demand projections is recommended prior to final funding determination.

In [11]:
# ---------------------------------------
# Export Reviewer Summary Workbook (XLSX) with formatting
# Creates:
# - Findings (review issues + adjustments)
# - Revised_CashFlows (revised benefit streams + PV)
# - Summary (Applicant vs Revised BCR at 7% and 3%)
# ---------------------------------------

from pathlib import Path
from openpyxl import load_workbook

# 1) Build Findings table (edit text if you want)
findings = pd.DataFrame(
    {
        "Category": ["Traffic Growth", "Value of Statistical Life", "Discount Rate Sensitivity"],
        "Applicant Assumption": ["4% annually", "$9.6M", "7% only"],
        "Reviewer Adjustment": ["1.5% annually", "Updated per current USDOT guidance (illustrative +10%)", "Added 3% sensitivity"],
        "Rationale": [
            "Growth not supported by corridor documentation",
            "Align valuation inputs with current federal guidance",
            "A-94 sensitivity analysis at 3% and 7%",
        ],
    }
)

# 2) Build Summary table
summary_rev = pd.DataFrame(
    {
        "Metric": [
            "Applicant PV Benefits (7%)",
            "Applicant PV Costs (7%)",
            "Applicant BCR (7%)",
            "Revised PV Benefits (7%)",
            "Revised PV Costs (7%)",
            "Revised BCR (7%)",
            "Revised BCR (3%)",
        ],
        "Value": [
            pv_benefit_app,
            pv_cost_app,
            bcr_app,
            pv_benefit_rev,
            pv_cost_rev,
            bcr_rev,
            bcr_3,
        ],
    }
)

# 3) Choose what columns to export for revised cash flows
revised_cols = [
    "year",
    "discount_factor",
    "total_cost",
    "safety_benefit",
    "travel_time_benefit",
    "voc_benefit",
    "emissions_benefit",
    "total_benefit",
    "pv_cost",
    "pv_benefit",
]
revised_cashflows = cashflows_rev[revised_cols].copy()

# 4) Write workbook
out_path = Path("reviewer_summary.xlsx")
with pd.ExcelWriter(out_path, engine="openpyxl") as writer:
    findings.to_excel(writer, sheet_name="Findings", index=False)
    summary_rev.to_excel(writer, sheet_name="Summary", index=False)
    revised_cashflows.to_excel(writer, sheet_name="Revised_CashFlows", index=False)

# 5) Apply formatting with openpyxl
wb = load_workbook(out_path)

# --- Format Summary sheet ---
ws_sum = wb["Summary"]
# Column B values formatting based on metric type
for r in range(2, ws_sum.max_row + 1):
    metric = ws_sum[f"A{r}"].value
    cell = ws_sum[f"B{r}"]
    if metric is None:
        continue
    if "PV" in metric:
        cell.number_format = '$#,##0'
    elif "BCR" in metric:
        cell.number_format = '0.00'

# --- Format Revised_CashFlows sheet ---
ws_cf = wb["Revised_CashFlows"]

# discount_factor column B
for r in range(2, ws_cf.max_row + 1):
    ws_cf[f"B{r}"].number_format = '0.0000'

# currency columns in Revised_CashFlows
currency_cols = [
    "total_cost",
    "safety_benefit",
    "travel_time_benefit",
    "voc_benefit",
    "emissions_benefit",
    "total_benefit",
    "pv_cost",
    "pv_benefit",
]

# Map based on revised_cashflows (the df actually exported)
for col_name in currency_cols:
    col_idx = revised_cashflows.columns.get_loc(col_name) + 1  # 1-based
    col_letter = ws_cf.cell(row=1, column=col_idx).column_letter
    for r in range(2, ws_cf.max_row + 1):
        ws_cf[f"{col_letter}{r}"].number_format = '$#,##0'

wb.save(out_path)

print(f"Saved reviewer summary workbook to: {out_path.resolve()}")

Saved reviewer summary workbook to: /home/kip/PyCharmMiscProject/reviewer_summary.xlsx
